In [21]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [22]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [23]:
#Query data from order_logs_snapshot

q = """
select yyyymmdd, epoch, data_orderId, data_propagationBatchId, data_riderId, eventType, hh, updated_epoch from raw.kafka_dispatch_propagation_immutable
where data_serviceDetailId = '5c53562fceb6fc9241980547'
and yyyymmdd = '20230620'
and hh in ('17', '18', '19', '20', '21')
and eventType = 'rider_acknowledged'
"""


#Load data into pandas table
df_propagation_copy = pd.read_sql(q, connection)

In [24]:
df_propagation = df_propagation_copy.copy()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch
0,20230620,1687278412659,6491d34ce205f470687a26d2,f3fb2ec9-f2cc-4c01-a6d7-a44450d7923e,613f49f4d087ea765b49f3b5,rider_acknowledged,21,1687278429430
1,20230620,1687277743199,6491d0af8b67de2627b5d85e,28a86ef5-994f-4be7-8b13-d486cf4f8f1e,61063ce674addcc0b9b92fde,rider_acknowledged,21,1687277744695
2,20230620,1687278384434,6491d330db4fa57679b2c7a2,f76b0254-c2fd-44e9-8045-15e336d0c99c,61e861c1dd30a4596ea296a0,rider_acknowledged,21,1687278471306
3,20230620,1687278197916,6491d275c1b4474a4f66f3e8,6176efaf-8496-4883-93c0-b8da99bf226d,62a0e8027d7967761be26026,rider_acknowledged,21,1687278204608
4,20230620,1687278021576,6491d1c5e205f470687a24f1,61c2a2e7-cd7d-4d4c-9e0e-474f8eb47659,636c927f50ecde7a3e03f0a3,rider_acknowledged,21,1687278025408


In [25]:
df_propagation = df_propagation.sort_values(by = ['data_orderId', 'updated_epoch', 'data_propagationBatchId'])
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch
1056206,20230620,1687265118904,648f489ad17502330558832b,93a5f33c-9895-4ae9-a0db-fd462e10f374,5df7762b0c84493417c6494d,rider_acknowledged,18,1687265124772
863392,20230620,1687265118904,648f489ad17502330558832b,93a5f33c-9895-4ae9-a0db-fd462e10f374,60e5b13909aa9954d741deec,rider_acknowledged,18,1687265125514
1134724,20230620,1687265118904,648f489ad17502330558832b,dcb17684-e075-4383-a66c-a04b153c90ec,601ee67ea6cc79618521e13d,rider_acknowledged,18,1687265255126
1326705,20230620,1687266323249,6490ebec2b1dca469bf18e7c,1d0c6e2b-2ba2-431a-9291-965eb64fe1f0,5e44bdd3cca9b8ffa203fb60,rider_acknowledged,18,1687266324865
1328877,20230620,1687266323249,6490ebec2b1dca469bf18e7c,1d0c6e2b-2ba2-431a-9291-965eb64fe1f0,5d385126cebab8653ab27bf9,rider_acknowledged,18,1687266324947


In [26]:
df_propagation['counter'] = (df_propagation['data_propagationBatchId'] != df_propagation['data_propagationBatchId'].shift()) | (df_propagation['data_orderId'] != df_propagation['data_orderId'].shift())
df_propagation['counter'] = df_propagation.groupby('data_orderId')['counter'].cumsum()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter
1056206,20230620,1687265118904,648f489ad17502330558832b,93a5f33c-9895-4ae9-a0db-fd462e10f374,5df7762b0c84493417c6494d,rider_acknowledged,18,1687265124772,1
863392,20230620,1687265118904,648f489ad17502330558832b,93a5f33c-9895-4ae9-a0db-fd462e10f374,60e5b13909aa9954d741deec,rider_acknowledged,18,1687265125514,1
1134724,20230620,1687265118904,648f489ad17502330558832b,dcb17684-e075-4383-a66c-a04b153c90ec,601ee67ea6cc79618521e13d,rider_acknowledged,18,1687265255126,2
1326705,20230620,1687266323249,6490ebec2b1dca469bf18e7c,1d0c6e2b-2ba2-431a-9291-965eb64fe1f0,5e44bdd3cca9b8ffa203fb60,rider_acknowledged,18,1687266324865,1
1328877,20230620,1687266323249,6490ebec2b1dca469bf18e7c,1d0c6e2b-2ba2-431a-9291-965eb64fe1f0,5d385126cebab8653ab27bf9,rider_acknowledged,18,1687266324947,1


In [27]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8, estimated_accept_to_pickup_distance
from orders.order_logs_immutable
where yyyymmdd = '20230620'
and service_obj_service_name = 'Auto'
and event_type in ('accepted', 'rider_busy', 'rider_reject')
and city_name = 'Bangalore'
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [28]:
df_apr = df_apr_copy.copy()
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,estimated_accept_to_pickup_distance
0,6491c63fc1b4474a4f66e4cb,NaN,8861892e0dfffff,12.995173,77.696249,Bangalore,8861892e0dfffff,8861892e0bfffff,12.988797,77.690054,12.993968,77.700668,210111,21,2100,Auto,21:00,2,20230620,new,None,1687275184753,rider_busy,61016a566c5e5f55b0bf05a3,None,8861892e0dfffff,1331.5
1,6491cb8ae205f470687a1d04,NaN,8861892467fffff,12.903978,77.648854,Bangalore,886189255bfffff,8861892097fffff,12.938552,77.695394,12.914673,77.653751,212346,21,2115,Auto,21:00,2,20230620,new,None,1687276584989,rider_busy,5ec8fb40dcc41b0bfd31cfe3,None,886189255bfffff,1727.2
2,6491c65afc04cd243acb1298,NaN,8861892e93fffff,12.983910,77.603540,Bangalore,8860145b41fffff,8860145b45fffff,12.986425,77.594925,12.978811,77.591583,210138,21,2100,Auto,21:00,2,20230620,new,None,1687275229738,rider_reject,6258ed3b8004cf014d66aaca,None,8860145b41fffff,1892.4
3,6491c668fc04cd243acb12b0,NaN,8861892589fffff,12.936285,77.607019,Bangalore,88618925d5fffff,88618925c7fffff,12.929644,77.615787,12.927850,77.605873,210152,21,2100,Auto,21:00,2,20230620,new,None,1687275186456,rider_busy,6303b24404cb1b1dea0dc5ae,None,88618925d5fffff,1491.6
4,6491c66a0c0f280f8d51820a,NaN,886189259dfffff,12.932575,77.589270,Bangalore,8861892595fffff,88618925c9fffff,12.916585,77.621930,12.942260,77.585122,210154,21,2100,Auto,21:00,2,20230620,new,None,1687275141144,rider_busy,62408ada59c221f6a5625f97,None,8861892595fffff,1421.3


In [29]:
df_apr_copy.shape

(1005809, 27)

In [30]:
df_apr.shape

(1005809, 27)

In [31]:
df_propagation = df_propagation.rename(columns = {'data_orderId':'order_id', 'data_riderId':'captain_id'})
df_propagation_filter = df_propagation[['order_id', 'captain_id', 'counter']]
df_propagation_filter = df_propagation_filter.drop_duplicates()
df_apr = df_apr.merge(df_propagation_filter, on = ['order_id', 'captain_id'], how = 'left')
df_accepted = df_apr[df_apr['event_type'] == 'accepted']
df_accepted.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,estimated_accept_to_pickup_distance,counter
32,6491c78936ad677d0648091f,NaN,8861892ea1fffff,13.014047,77.639981,Bangalore,8861892ea7fffff,8861892c1bfffff,13.027398,77.638041,13.021091,77.636940,210641,21,2100,Auto,21:00,2,20230620,onTheWay,None,1687275471711,accepted,6245c34b854be43445ac3503,None,8861892ea7fffff,1264.2,2.0
88,6491ca4f2b1dca469bf32ea5,NaN,8861892595fffff,12.943582,77.589061,Bangalore,8861892591fffff,88618925d5fffff,12.929530,77.603097,12.936036,77.581024,211831,21,2115,Auto,21:00,2,20230620,onTheWay,None,1687276288972,accepted,63d0c0fb10097ee5b323c4c4,None,8861892591fffff,1886.7,5.0
115,6491cbb0b2f49b01869c8d34,NaN,8861892e9bfffff,12.975291,77.606625,Bangalore,88618925a5fffff,8861892ed3fffff,12.971303,77.618573,12.971011,77.607127,212424,21,2115,Auto,21:00,2,20230620,onTheWay,None,1687276481862,accepted,610ba1d1e38015fc046e6512,None,88618925a5fffff,643.3,1.0
141,6491d2f1a6877d27a7b10f42,NaN,8861892e33fffff,13.012045,77.667787,Bangalore,8861892e31fffff,8861892f5bfffff,13.016371,77.716942,13.009197,77.673737,215521,21,2145,Auto,21:59,2,20230620,onTheWay,None,1687278329554,accepted,647316449fbf21cd3cac4d3a,None,8861892e31fffff,961.9,1.0
145,6491d323c1b4474a4f66f4ce,NaN,886189242dfffff,12.911737,77.644561,Bangalore,8861892553fffff,88618925c9fffff,12.917216,77.624474,12.921018,77.645111,215611,21,2145,Auto,21:59,2,20230620,onTheWay,None,1687278379388,accepted,5e6b76188e48c09c759fc444,None,8861892553fffff,1324.2,1.0


In [32]:
q = """
select * from datasets.hex_adjacency
where city_name = 'bangalore'
"""

df_adjacency_copy = pd.read_sql(q, connection)
df_adjacency_copy.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,multicast_config,city_name
0,8861892c3bfffff-Monday-rest_evening,Monday,rest_evening,8861892c3bfffff,"['8861892c39fffff', '8861892c33fffff', '886189...","['8861892c37fffff', '8861892c39fffff', '886189...",2022_reset,22:00,23:59,572ca7ff116b5db3057bd814,20211022,multicast,"[[0,5],[5,10],[10,15]]",5,57370b61a6855d70057417d1,None,3,"[[-1,-0.75],[-0.75,0],[0,0.75],[0.75,1]]",False,False,5,,0.0,None,bangalore
1,88601441edfffff-Thursday-morning_peak,Thursday,morning_peak,88601441edfffff,"['8860144133fffff', '88601441edfffff', '886014...","['88601441e1fffff', '8860144117fffff', '886014...",2022_reset,08:00,12:00,572ca7ff116b5db3057bd814,20211022,multicast,"[[0,5],[5,10],[10,15]]",4,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,None,bangalore
2,8860145067fffff-Sunday-rest_evening,Sunday,rest_evening,8860145067fffff,"['8860145065fffff', '886014502dfffff', '886014...","['8860145063fffff', '8860145151fffff', '886014...",2022_reset,22:00,23:59,572ca7ff116b5db3057bd814,20211022,multicast,"[[0,5],[5,10],[10,15]]",5,57370b61a6855d70057417d1,None,3,"[[-1,-0.75],[-0.75,0],[0,0.75],[0.75,1]]",False,False,5,,0.0,None,bangalore
3,886189351bfffff-Sunday-evening_peak,Sunday,evening_peak,886189351bfffff,"['8861893511fffff', '886189351bfffff', '886189...","['8861893513fffff', '8861893515fffff', '886189...",reset_auto_adj,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,False,5,,0.7,None,bangalore
4,8860145a5dfffff-Friday-rest_evening,Friday,rest_evening,8860145a5dfffff,"['8860145a51fffff', '88618924b1fffff', '886014...","['8860145a51fffff', '88618924b1fffff', '886014...",increased_search_space_ATL,22:00,23:59,572ca7ff116b5db3057bd814,20211022,multicast,"[[0,5],[5,10],[10,15]]",5,57370b61a6855d70057417d1,None,3,"[[-1,-0.75],[-0.75,0],[0,0.75],[0.75,1]]",False,False,5,,0.0,None,bangalore


In [33]:
df_adjacency = df_adjacency_copy.copy()
df_adjacency.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,multicast_config,city_name
0,8861892c3bfffff-Monday-rest_evening,Monday,rest_evening,8861892c3bfffff,"['8861892c39fffff', '8861892c33fffff', '886189...","['8861892c37fffff', '8861892c39fffff', '886189...",2022_reset,22:00,23:59,572ca7ff116b5db3057bd814,20211022,multicast,"[[0,5],[5,10],[10,15]]",5,57370b61a6855d70057417d1,None,3,"[[-1,-0.75],[-0.75,0],[0,0.75],[0.75,1]]",False,False,5,,0.0,None,bangalore
1,88601441edfffff-Thursday-morning_peak,Thursday,morning_peak,88601441edfffff,"['8860144133fffff', '88601441edfffff', '886014...","['88601441e1fffff', '8860144117fffff', '886014...",2022_reset,08:00,12:00,572ca7ff116b5db3057bd814,20211022,multicast,"[[0,5],[5,10],[10,15]]",4,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,None,bangalore
2,8860145067fffff-Sunday-rest_evening,Sunday,rest_evening,8860145067fffff,"['8860145065fffff', '886014502dfffff', '886014...","['8860145063fffff', '8860145151fffff', '886014...",2022_reset,22:00,23:59,572ca7ff116b5db3057bd814,20211022,multicast,"[[0,5],[5,10],[10,15]]",5,57370b61a6855d70057417d1,None,3,"[[-1,-0.75],[-0.75,0],[0,0.75],[0.75,1]]",False,False,5,,0.0,None,bangalore
3,886189351bfffff-Sunday-evening_peak,Sunday,evening_peak,886189351bfffff,"['8861893511fffff', '886189351bfffff', '886189...","['8861893513fffff', '8861893515fffff', '886189...",reset_auto_adj,17:00,22:00,572ca7ff116b5db3057bd814,20211215,multicast,"[[0,6],[6,12],[12,18]]",20,5c53562fceb6fc9241980547,None,3,"[[-1,-0.4], [-0.4,1]]",True,False,5,,0.7,None,bangalore
4,8860145a5dfffff-Friday-rest_evening,Friday,rest_evening,8860145a5dfffff,"['8860145a51fffff', '88618924b1fffff', '886014...","['8860145a51fffff', '88618924b1fffff', '886014...",increased_search_space_ATL,22:00,23:59,572ca7ff116b5db3057bd814,20211022,multicast,"[[0,5],[5,10],[10,15]]",5,57370b61a6855d70057417d1,None,3,"[[-1,-0.75],[-0.75,0],[0,0.75],[0.75,1]]",False,False,5,,0.0,None,bangalore


In [34]:
df_adjacency_filter = df_adjacency[(df_adjacency['time_period'] == 'evening_peak') & (df_adjacency['service_detail_id'] == '5c53562fceb6fc9241980547')]
df_adjacency_filter = df_adjacency_filter[['source_hex', 'multicast_max_batch_size']]
df_adjacency_filter = df_adjacency_filter.drop_duplicates()
df_adjacency_filter = df_adjacency_filter.rename(columns = {'source_hex':'pickup_location_hex_8'})
df_adjacency_filter.head()

,pickup_location_hex_8,multicast_max_batch_size
3,886189351bfffff,20
51,886014425dfffff,20
67,88618922a9fffff,20
123,88601451c9fffff,20
171,8861893421fffff,20


In [35]:
df_accepted = df_accepted.merge(df_adjacency_filter, on = 'pickup_location_hex_8', how = 'left')
df_accepted.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,estimated_accept_to_pickup_distance,counter,multicast_max_batch_size
0,6491c78936ad677d0648091f,NaN,8861892ea1fffff,13.014047,77.639981,Bangalore,8861892ea7fffff,8861892c1bfffff,13.027398,77.638041,13.021091,77.636940,210641,21,2100,Auto,21:00,2,20230620,onTheWay,None,1687275471711,accepted,6245c34b854be43445ac3503,None,8861892ea7fffff,1264.2,2.0,20
1,6491ca4f2b1dca469bf32ea5,NaN,8861892595fffff,12.943582,77.589061,Bangalore,8861892591fffff,88618925d5fffff,12.929530,77.603097,12.936036,77.581024,211831,21,2115,Auto,21:00,2,20230620,onTheWay,None,1687276288972,accepted,63d0c0fb10097ee5b323c4c4,None,8861892591fffff,1886.7,5.0,20
2,6491cbb0b2f49b01869c8d34,NaN,8861892e9bfffff,12.975291,77.606625,Bangalore,88618925a5fffff,8861892ed3fffff,12.971303,77.618573,12.971011,77.607127,212424,21,2115,Auto,21:00,2,20230620,onTheWay,None,1687276481862,accepted,610ba1d1e38015fc046e6512,None,88618925a5fffff,643.3,1.0,20
3,6491d2f1a6877d27a7b10f42,NaN,8861892e33fffff,13.012045,77.667787,Bangalore,8861892e31fffff,8861892f5bfffff,13.016371,77.716942,13.009197,77.673737,215521,21,2145,Auto,21:59,2,20230620,onTheWay,None,1687278329554,accepted,647316449fbf21cd3cac4d3a,None,8861892e31fffff,961.9,1.0,20
4,6491d323c1b4474a4f66f4ce,NaN,886189242dfffff,12.911737,77.644561,Bangalore,8861892553fffff,88618925c9fffff,12.917216,77.624474,12.921018,77.645111,215611,21,2145,Auto,21:59,2,20230620,onTheWay,None,1687278379388,accepted,5e6b76188e48c09c759fc444,None,8861892553fffff,1324.2,1.0,20


In [36]:
batch_wise_acceptance = df_accepted.groupby(['multicast_max_batch_size', 'counter']).size().reset_index()
batch_wise_acceptance = batch_wise_acceptance.rename(columns = {0:'accepted_pings'})
batch_wise_acceptance2 = batch_wise_acceptance.groupby(['multicast_max_batch_size'])['accepted_pings'].sum().reset_index()
batch_wise_acceptance2 = batch_wise_acceptance2.rename(columns = {'accepted_pings':'total_accepted_pings'})
batch_wise_acceptance = batch_wise_acceptance.merge(batch_wise_acceptance2, on = 'multicast_max_batch_size', how = 'left')
batch_wise_acceptance

,multicast_max_batch_size,counter,accepted_pings,total_accepted_pings
0,20,1.0,28355,47537
1,20,2.0,9284,47537
2,20,3.0,4483,47537
3,20,4.0,2662,47537
4,20,5.0,1630,47537
5,20,6.0,528,47537
6,20,7.0,234,47537
7,20,8.0,137,47537
8,20,9.0,97,47537
9,20,10.0,64,47537


In [37]:
batch_wise_acceptance.to_csv('bangalore_batch_wise_acceptance.csv', index = False)

In [17]:
batch_wise_acceptance = pd.DataFrame(df_accepted['counter'].value_counts().reset_index())
batch_wise_acceptance.columns = ['batch_number', 'accepted_pings']
batch_wise_acceptance['total_accepted_pings'] = df_accepted.shape[0]
batch_wise_acceptance['%_of_accepted_pings'] = round(batch_wise_acceptance['accepted_pings']/batch_wise_acceptance['total_accepted_pings'], 4)
batch_wise_acceptance

,batch_number,accepted_pings,total_accepted_pings,%_of_accepted_pings
0,1.0,28743,47219,0.6087
1,2.0,9069,47219,0.1921
2,3.0,4313,47219,0.0913
3,4.0,2445,47219,0.0518
4,5.0,1467,47219,0.0311
5,6.0,463,47219,0.0098
6,7.0,234,47219,0.0050
7,8.0,141,47219,0.0030
8,9.0,84,47219,0.0018
9,10.0,47,47219,0.0010


In [20]:
batch_wise_acceptance.to_csv('bangalore_batch_wise_acceptance.csv', index = False)

In [17]:
df_propagation.head(20)

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter
393665,20230510,1683726307914,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60e99160e5cb35adea5900ff,rider_acknowledged,19,1683726307914,1
543576,20230510,1683726307998,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,5ebaca21350aa75c87bb7f34,rider_acknowledged,19,1683726307998,1
398398,20230510,1683726309114,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,632bf4134bdbff70462cb037,rider_acknowledged,19,1683726309114,1
414195,20230510,1683726309676,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,6319d9882badab5c7d3a3a5e,rider_acknowledged,19,1683726309676,1
509157,20230510,1683726310208,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60295c2a19e4984b6c1e9771,rider_acknowledged,19,1683726310208,1
391033,20230510,1683726917417,645b0c66908b9d588b5dc66f,fe0cf86e-ce13-4337-af6d-6afb5855c10e,642bffc70e17959cf78df534,rider_acknowledged,19,1683726917417,1
192147,20230510,1683722143183,645b2178af0d3209d269c3df,ebe83566-7f3a-4ebd-972a-bd9518290712,60f121df8bb15d81b65193f2,rider_acknowledged,18,1683722143183,1
549634,20230510,1683730536871,645b39b670e5ce05969bf500,8c3ec9be-8897-4e47-b353-6497fd241810,62ac9b667d79671ecaeba24e,rider_acknowledged,20,1683730536871,1
7989,20230510,1683730692786,645b39b670e5ce05969bf500,a7480d15-da57-4ff1-a354-605b2bc41723,6436b932303d7e7a54697cf4,rider_acknowledged,20,1683730692786,2
254495,20230510,1683730581854,645b39b670e5ce05969bf500,d17d70cd-14b1-497a-bab0-8211fb07640b,62ac9b667d79671ecaeba24e,rider_acknowledged,20,1683730581854,3


In [18]:
df_propagation[df_propagation['data_propagationBatchId'] == 'b8f09de1-5612-4e26-afdd-9b838e627092']

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter
393665,20230510,1683726307914,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60e99160e5cb35adea5900ff,rider_acknowledged,19,1683726307914,1
543576,20230510,1683726307998,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,5ebaca21350aa75c87bb7f34,rider_acknowledged,19,1683726307998,1
398398,20230510,1683726309114,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,632bf4134bdbff70462cb037,rider_acknowledged,19,1683726309114,1
414195,20230510,1683726309676,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,6319d9882badab5c7d3a3a5e,rider_acknowledged,19,1683726309676,1
509157,20230510,1683726310208,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60295c2a19e4984b6c1e9771,rider_acknowledged,19,1683726310208,1


In [19]:
captains_pinged_per_batch = df_propagation.groupby(['data_propagationBatchId'])['data_riderId'].nunique().reset_index()
captains_pinged_per_batch = captains_pinged_per_batch.rename(columns = {'data_riderId':'captains_pinged'})
df_propagation_unique = df_propagation[['data_propagationBatchId', 'counter']]
df_propagation_unique = df_propagation_unique.drop_duplicates()
captains_pinged_per_batch = captains_pinged_per_batch.merge(df_propagation_unique, on = 'data_propagationBatchId', how = 'left')
captains_pinged_per_batch.head()

,data_propagationBatchId,captains_pinged,counter
0,00000fcb-368c-4802-9bcd-94bc5621dacf,1,1
1,00003e40-737e-4dbc-996a-4274bc48d782,2,1
2,00005ce3-c170-4138-ba7d-2505706c7a0e,2,1
3,000077c0-fd84-4fb4-b588-52489a01d9b8,10,1
4,000092fc-2f8b-492a-af0b-a0e76e7866e9,1,1


In [20]:
captains_pinged_per_batch.groupby(['counter'])['captains_pinged'].mean().reset_index()

,counter,captains_pinged
0,1,1.905736
1,2,1.782025
2,3,1.762390
3,4,1.801321
4,5,1.858209
5,6,1.688690
6,7,1.731382
7,8,1.753266
8,9,1.717557
9,10,1.775281


In [18]:
df_propagation['counter'] = (df_propagation['data_propagationBatchId'] != df_propagation['data_propagationBatchId'].shift()) | (df_propagation['data_orderId'] != df_propagation['data_orderId'].shift())
df_propagation['counter'] = df_propagation.groupby('data_orderId')['counter'].cumsum()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,counter
400155,20230510,1683726307914,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60e99160e5cb35adea5900ff,rider_acknowledged,19,1
242489,20230510,1683726307998,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,5ebaca21350aa75c87bb7f34,rider_acknowledged,19,1
240032,20230510,1683726309114,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,632bf4134bdbff70462cb037,rider_acknowledged,19,1
229761,20230510,1683726309676,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,6319d9882badab5c7d3a3a5e,rider_acknowledged,19,1
482423,20230510,1683726310208,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60295c2a19e4984b6c1e9771,rider_acknowledged,19,1


In [19]:
df_propagation_filter = df_propagation[['data_orderId', 'data_riderId', 'counter']]
df_propagation_filter.head()

,data_orderId,data_riderId,counter
400155,645a6ddc8b51b47a8f1d567f,60e99160e5cb35adea5900ff,1
242489,645a6ddc8b51b47a8f1d567f,5ebaca21350aa75c87bb7f34,1
240032,645a6ddc8b51b47a8f1d567f,632bf4134bdbff70462cb037,1
229761,645a6ddc8b51b47a8f1d567f,6319d9882badab5c7d3a3a5e,1
482423,645a6ddc8b51b47a8f1d567f,60295c2a19e4984b6c1e9771,1


In [20]:
#Merge this on df
df_propagation_filter = df_propagation_filter.rename(columns = {'data_orderId':'order_id', 'data_riderId':'captain_id'})
df = df.merge(df_propagation_filter, on = ['order_id', 'captain_id'], how = 'left')
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,counter
0,645bae48bc677762c758e7c3,NaN,Bangalore,8861892591fffff,12.930825,77.580872,201632,20,2015,Auto,20:00,3,20230510,expired,None,7.0,"[""60ed2fb613ae1d748eb5528a"",""6013ae76edd74b702...",8861892591fffff,62a072577d796770ede1fe5f,1060.2,2.0
1,645bae4bde453b02ef33750e,NaN,Bangalore,8861892513fffff,12.931752,77.634494,201635,20,2015,Auto,20:00,3,20230510,expired,None,2.0,"[""60f40f138f3479e752195633"",""63b683460e10cd5a3...",8861892513fffff,63d405e015a98c9f1f0db9af,1882.5,6.0
2,645bae58888565720ae938e6,NaN,Bangalore,8861892433fffff,12.906545,77.606756,201648,20,2015,Auto,20:00,3,20230510,customerCancelled,order cancelled before rider accepted,2.0,"[""60685591a2f9c56b225129b7"",""62578fdd00b5f49e5...",8861892433fffff,,NaN,NaN
3,645baf8b888565720ae93b0d,0.438,Bangalore,88618925ddfffff,12.921208,77.604073,202155,20,2015,Auto,20:00,3,20230510,dropped,None,2.0,"[""62921b26444fa9ba4f782075"",""610a4a8a06f56f41f...",88618925ddfffff,631b7e788276172360bbb215,NaN,1.0
4,645bb05bb70c5170c58d75f8,NaN,Bangalore,88618925c1fffff,12.923099,77.620018,202523,20,2015,Auto,20:00,3,20230510,expired,None,0.0,"[""6442088be2719e34f0671600"",""62578fdd00b5f49e5...",88618925c1fffff,6442088be2719e34f0671600,1122.7,1.0


In [21]:
df['counter'] = df['counter'].fillna(1)
df['counter_group'] = np.where(df['counter'] >= 5, '5 or more batches', df['counter'])
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,counter,counter_group
0,645bae48bc677762c758e7c3,NaN,Bangalore,8861892591fffff,12.930825,77.580872,201632,20,2015,Auto,20:00,3,20230510,expired,None,7.0,"[""60ed2fb613ae1d748eb5528a"",""6013ae76edd74b702...",8861892591fffff,62a072577d796770ede1fe5f,1060.2,2.0,2.0
1,645bae4bde453b02ef33750e,NaN,Bangalore,8861892513fffff,12.931752,77.634494,201635,20,2015,Auto,20:00,3,20230510,expired,None,2.0,"[""60f40f138f3479e752195633"",""63b683460e10cd5a3...",8861892513fffff,63d405e015a98c9f1f0db9af,1882.5,6.0,5 or more batches
2,645bae58888565720ae938e6,NaN,Bangalore,8861892433fffff,12.906545,77.606756,201648,20,2015,Auto,20:00,3,20230510,customerCancelled,order cancelled before rider accepted,2.0,"[""60685591a2f9c56b225129b7"",""62578fdd00b5f49e5...",8861892433fffff,,NaN,1.0,1.0
3,645baf8b888565720ae93b0d,0.438,Bangalore,88618925ddfffff,12.921208,77.604073,202155,20,2015,Auto,20:00,3,20230510,dropped,None,2.0,"[""62921b26444fa9ba4f782075"",""610a4a8a06f56f41f...",88618925ddfffff,631b7e788276172360bbb215,NaN,1.0,1.0
4,645bb05bb70c5170c58d75f8,NaN,Bangalore,88618925c1fffff,12.923099,77.620018,202523,20,2015,Auto,20:00,3,20230510,expired,None,0.0,"[""6442088be2719e34f0671600"",""62578fdd00b5f49e5...",88618925c1fffff,6442088be2719e34f0671600,1122.7,1.0,1.0


In [22]:
#Filter only on the hexes needed
df_filter_level1 = df
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].astype(str).apply(lambda x: x[6:])

In [23]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['order_status'] == 'customerCancelled',
 np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

In [24]:
#Check if the order is expiry_mapped or expiry_unmapped
df_filter_level1['expiry_type'] = np.where(df_filter_level1['order_status'] == 'expired',
 np.where(df_filter_level1['map_riders'] == '[]', 'expiry_unmapped', 'expiry_mapped'), 'no_expiry')
df_filter_level1['final_order_status'] = np.where(df_filter_level1['order_status'] == 'customerCancelled', df_filter_level1['new_cancel_reason'],
np.where(df_filter_level1['order_status'] == 'expired', df_filter_level1['expiry_type'], df_filter_level1['order_status']))

In [32]:
order_status = df_filter_level1[df_filter_level1['counter_group'] == '2.0']
order_status = order_status[['order_id', 'final_order_status']]

In [43]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_id, estimated_accept_to_pickup_distance, updated_epoch, event_type
from orders.order_logs_immutable
where yyyymmdd = '20230510'
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
and pickup_location_hex_8 in ('8861892433fffff', '886189251bfffff', '88618925c3fffff', '8861892589fffff', '886189242dfffff', '8861892421fffff', '88618925d3fffff',
 '88618925cdfffff', '88618925d1fffff', '886189255bfffff', '8861892553fffff', '88618925d5fffff', '88618925e3fffff', '88618925ebfffff', '88618924adfffff',
   '8861892513fffff', '88618924e7fffff', '88618925c5fffff', '88618925dbfffff', '88618925ddfffff', '88618925c7fffff', '88618924a5fffff', '88618924e5fffff',
     '88618925c9fffff', '8861892599fffff', '88618925d9fffff', '8861892437fffff', '88618925e9fffff', '88618925d7fffff', '88618925c1fffff', '8861892591fffff',
       '88618925cbfffff', '8861892581fffff', '8861892431fffff', '886189259dfffff', '886189258dfffff', '886189258bfffff', '886189259bfffff', '8861892435fffff',
         '8861892517fffff', '88618925e1fffff', '88618925edfffff', '88618925e7fffff')
and hour in ('17', '18', '19', '20', '21')
and event_type in ('accepted', 'rider_busy', 'rider_reject')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [44]:
df_copy = df_copy.merge(order_status, on = 'order_id', how = 'left')
df_copy.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,updated_epoch,event_type,final_order_status
0,645bc55913c08a4ec9300ca1,NaN,Bangalore,8861892437fffff,12.910453,77.612503,215457,21,2145,Auto,21:59,3,20230510,new,None,2.0,"[""629f19df1b57e4adbae9f447"",""61fe1f9c725c4cea8...",8861892437fffff,62cbb9b801d28934302b9682,1214.4,1683735926040,rider_busy,expiry_mapped
1,645bba478dbcb75a89aaf31b,NaN,Bangalore,8861892589fffff,12.933781,77.608307,210743,21,2100,Auto,21:00,3,20230510,new,None,0.0,"[""62d3f19e32a921678d77579e"",""642e67c41ecc3a9b6...",8861892589fffff,62d3f19e32a921678d77579e,437.9,1683733123970,rider_busy,NaN
2,645bc5db612b712367f9b7c6,NaN,Bangalore,88618925c7fffff,12.935354,77.614131,215707,21,2145,Auto,21:59,3,20230510,new,None,3.0,"[""60e94a1fac6ff47ed860ad5f"",""62d4cf8a24fad3312...",88618925c7fffff,6151938cf2602c3cb62cabc4,953.6,1683736130433,rider_reject,NaN
3,645bc5f3612b712367f9b7ee,NaN,Bangalore,88618925d7fffff,12.929103,77.591507,215731,21,2145,Auto,21:59,3,20230510,new,None,0.0,"[""5e5503e9dcc6a7fda2a2e67f"",""60e44ebf09aa996cb...",88618925d7fffff,60e44ebf09aa996cbb41180b,1090.9,1683736141453,rider_busy,NaN
4,645bb892888565720ae94a6f,NaN,Bangalore,8861892c19fffff,13.030947,77.648964,210026,21,2100,Auto,21:00,3,20230510,new,None,1.0,"[""62d793f9ceb77701571006bc"",""62b1760317af9a1ea...",88618925e7fffff,6295caae74c07130cc508775,589.7,1683732663471,rider_busy,expiry_mapped


In [45]:
df_copy = df_copy.dropna()
df_copy.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,updated_epoch,event_type,final_order_status
724,645bb146751ce46ce03ec71d,0.763,Bangalore,88618925e9fffff,12.939100,77.626884,202918,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,3.0,"[""62578fdd00b5f49e57ad65ea"",""5cf27fe9ca6e29211...",88618925e9fffff,63e3885d052c8a26c7edbc1e,776.4,1683731007711,rider_busy,COBRA
1110,645b8f5c3f9e605f9192f02d,1.925,Bangalore,8861892421fffff,12.909473,77.636635,180436,18,1800,Auto,18:00,3,20230510,new,Asked to buy items,7.0,"[""5de5d28eac56a62ec5e6185a"",""61d2e630de3a0254a...",886189242dfffff,60d2e01a05468956fbddd94a,1428.8,1683723161695,rider_busy,COBRA
1950,645b89a7b70c5170c58d26e5,0.757,Bangalore,88618925ebfffff,12.942251,77.621309,174015,17,1730,Auto,17:59,3,20230510,new,Unable to swipe Arrived,4.0,"[""5f1f96fdea5a8ba56679a603"",""623192433a5bed58b...",88618925ebfffff,63a5698d64e8005ff3fcd475,723.0,1683721181739,rider_reject,expiry_mapped
2963,645b963770e5ce05969cb64d,1.849,Bangalore,88618924a5fffff,12.917027,77.582852,183351,18,1830,Auto,18:59,3,20230510,new,Long Pickup,7.0,"[""61ef89c41772e473ff144182"",""60fd0164fe2f60ae4...",88618924a5fffff,6116409ef76f8bd4df482a2f,939.4,1683724042726,rider_busy,expiry_mapped
3054,645ba0a5de453b02ef335a6f,1.411,Bangalore,886189251bfffff,12.925212,77.637230,191821,19,1915,Auto,19:00,3,20230510,new,Long Pickup,6.0,"[""623c2999d26cb255e872d4ff"",""6449f5248fac8ad0b...",886189251bfffff,623c2999d26cb255e872d4ff,466.1,1683726895887,rider_busy,COBRA


In [46]:
df_copy2 = df_copy[df_copy['order_id'].isin(['645bc2a9ec315d1c8d2c0a7d', '645b995ec78d275898f7b767', '645baf766a343d115bead631'])]
df_copy2['timestamp'] = df_copy2['updated_epoch'].apply(lambda x: datetime.fromtimestamp(x/1000))
df_copy2 = df_copy2.sort_values(by = ['order_id', 'updated_epoch'])
df_copy2

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,updated_epoch,event_type,final_order_status,timestamp
34708,645b995ec78d275898f7b767,0.060,Bangalore,8861892553fffff,12.916032,77.651622,184718,18,1845,Auto,18:59,3,20230510,new,Drop location denied,2.0,"[""5e49f514b3394b0f2f2a3656"",""640185a7f22eada02...",8861892553fffff,616cf4654c6ba1f7ced8305a,681.4,1683724901048,rider_reject,dropped,2023-05-10 18:51:41.048
106595,645b995ec78d275898f7b767,0.060,Bangalore,8861892553fffff,12.916032,77.651622,184718,18,1845,Auto,18:59,3,20230510,onTheWay,Drop location denied,2.0,"[""5e49f514b3394b0f2f2a3656"",""640185a7f22eada02...",8861892553fffff,5e49f514b3394b0f2f2a3656,177.8,1683724905431,accepted,dropped,2023-05-10 18:51:45.431
98465,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,62a306a81284ca0edd09f4cf,968.1,1683730376738,rider_reject,COBRA,2023-05-10 20:22:56.738
97277,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,61cc08719ba37202862da8bc,1063.6,1683730380880,rider_reject,COBRA,2023-05-10 20:23:00.880
24471,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,6108eb0fb8bce11a8a1256b3,1913.8,1683730385758,rider_reject,COBRA,2023-05-10 20:23:05.758
32063,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,5be26f8f2b706c16e077f1cc,769.5,1683730386927,rider_reject,COBRA,2023-05-10 20:23:06.927
129617,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,60fbb2d9fe2f607ffe869f45,982.5,1683730394652,rider_reject,COBRA,2023-05-10 20:23:14.652
25280,645bc2a9ec315d1c8d2c0a7d,0.742,Bangalore,88618925e1fffff,12.945374,77.626076,214329,21,2130,Auto,21:59,3,20230510,new,Long Pickup,3.0,"[""62aeeeb267cffab13cea2c33"",""60fa8812d417d4a5a...",88618925e1fffff,6149fa01de864c3f3a034fe1,332.0,1683735350385,rider_reject,expiry_mapped,2023-05-10 21:45:50.385
115031,645bc2a9ec315d1c8d2c0a7d,0.742,Bangalore,88618925e1fffff,12.945374,77.626076,214329,21,2130,Auto,21:59,3,20230510,new,Long Pickup,3.0,"[""62aeeeb267cffab13cea2c33"",""60fa8812d417d4a5a...",88618925e1fffff,60fa8812d417d4a5a57dfa33,866.4,1683735378636,rider_reject,expiry_mapped,2023-05-10 21:46:18.636
88896,645bc2a9ec315d1c8d2c0a7d,0.742,Bangalore,88618925e1fffff,12.945374,77.626076,214329,21,2130,Auto,21:59,3,20230510,new,Long Pickup,3.0,"[""62aeeeb267cffab13cea2c33"",""610295906cd49257e...",88618925e1fffff,60e42d4209aa990f9b40fa77,1725.3,1683735422658,rider_busy,expiry_mapped,2023-05-10 21:47:02.658


In [47]:
df_copy2.to_csv('sample_orders.csv', index = False)

In [29]:
df_filter_level1

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,counter,counter_group,count,date,new_cancel_reason,expiry_type,final_order_status
0,645bae48bc677762c758e7c3,NaN,Bangalore,8861892591fffff,12.930825,77.580872,201632,20,2015,Auto,20:00,3,20230510,expired,None,7.0,"[""60ed2fb613ae1d748eb5528a"",""6013ae76edd74b702...",8861892591fffff,62a072577d796770ede1fe5f,1060.2,2.0,2.0,1,10,no_cancellation,expiry_mapped,expiry_mapped
1,645bae4bde453b02ef33750e,NaN,Bangalore,8861892513fffff,12.931752,77.634494,201635,20,2015,Auto,20:00,3,20230510,expired,None,2.0,"[""60f40f138f3479e752195633"",""63b683460e10cd5a3...",8861892513fffff,63d405e015a98c9f1f0db9af,1882.5,6.0,5 or more batches,1,10,no_cancellation,expiry_mapped,expiry_mapped
2,645bae58888565720ae938e6,NaN,Bangalore,8861892433fffff,12.906545,77.606756,201648,20,2015,Auto,20:00,3,20230510,customerCancelled,order cancelled before rider accepted,2.0,"[""60685591a2f9c56b225129b7"",""62578fdd00b5f49e5...",8861892433fffff,,NaN,1.0,1.0,1,10,COBRA,no_expiry,COBRA
3,645baf8b888565720ae93b0d,0.438,Bangalore,88618925ddfffff,12.921208,77.604073,202155,20,2015,Auto,20:00,3,20230510,dropped,None,2.0,"[""62921b26444fa9ba4f782075"",""610a4a8a06f56f41f...",88618925ddfffff,631b7e788276172360bbb215,NaN,1.0,1.0,1,10,no_cancellation,no_expiry,dropped
4,645bb05bb70c5170c58d75f8,NaN,Bangalore,88618925c1fffff,12.923099,77.620018,202523,20,2015,Auto,20:00,3,20230510,expired,None,0.0,"[""6442088be2719e34f0671600"",""62578fdd00b5f49e5...",88618925c1fffff,6442088be2719e34f0671600,1122.7,1.0,1.0,1,10,no_cancellation,expiry_mapped,expiry_mapped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30305,645b97f4c78d275898f7b451,NaN,Bangalore,88618924e7fffff,12.904047,77.588232,184116,18,1830,Auto,18:59,3,20230510,customerCancelled,order cancelled before rider accepted,3.0,"[""5c7bba9f875c9853625f6f1b"",""60f165268f34791bc...",88618924e7fffff,5c7bba9f875c9853625f6f1b,533.2,1.0,1.0,1,10,COBRA,no_expiry,COBRA
30306,645b9a33c78d275898f7b928,NaN,Bangalore,88618925c9fffff,12.916768,77.623550,185051,18,1845,Auto,18:59,3,20230510,expired,None,2.0,"[""61067df8a31862ce6b59c3bf""]",88618925c9fffff,6222fe71c1a4e4e1e4645af4,1954.5,3.0,3.0,1,10,no_cancellation,expiry_mapped,expiry_mapped
30307,645b98276a343d115beaa7cc,NaN,Bangalore,88618925ebfffff,12.940269,77.620054,184207,18,1830,Auto,18:59,3,20230510,customerCancelled,order cancelled before rider accepted,2.0,"[""623192433a5bed58b8bec937"",""60efd298d417d4cd4...",88618925ebfffff,643106272548e37cc9e53fa1,1295.4,2.0,2.0,1,10,COBRA,no_expiry,COBRA
30308,645b9830ec315d1c8d2bba01,NaN,Bangalore,88618925d9fffff,12.914090,77.599678,184216,18,1830,Auto,18:59,3,20230510,expired,None,2.0,"[""615163be90fbcb7e26d0ac55"",""60fd0164fe2f60ae4...",88618925d1fffff,6116409ef76f8bd4df482a2f,1237.6,4.0,4.0,1,10,no_cancellation,expiry_mapped,expiry_mapped


In [25]:
#Metrics_pivot
metrics_pivot = pd.pivot_table(df_filter_level1, index = ['yyyymmdd', 'counter_group'], columns = 'final_order_status',
 values = 'count', aggfunc = 'sum', fill_value=0).reset_index().rename_axis(None, axis=1)
metrics_pivot['total'] = metrics_pivot[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expiry_mapped', 'expiry_unmapped']].sum(axis = 1)
metrics_pivot['OCARA%'] = round(metrics_pivot['OCARA']/metrics_pivot['total'], 4)
metrics_pivot['COBRA%'] = round(metrics_pivot['COBRA']/metrics_pivot['total'], 4)
metrics_pivot['COBRM%'] = round(metrics_pivot['COBRM']/metrics_pivot['total'], 4)
metrics_pivot['G2N'] = round(metrics_pivot['dropped']/metrics_pivot['total'], 4)
metrics_pivot['expiry_mapped%'] = round(metrics_pivot['expiry_mapped']/metrics_pivot['total'], 4)
if 'expiry_unmapped' in metrics_pivot.columns:
    metrics_pivot['Stockout%'] = round(metrics_pivot['expiry_unmapped']/metrics_pivot['total'], 4)
else:
    metrics_pivot['Stockout%'] = 0
metrics_df = metrics_pivot[['yyyymmdd', 'counter_group', 'OCARA%', 'COBRA%', 'COBRM%', 'G2N', 'expiry_mapped%', 'Stockout%', 'total']]
metrics_df

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total
0,20230510,1.0,0.0252,0.5712,0.002,0.0996,0.3015,0.0002,9967
1,20230510,2.0,0.0412,0.5027,0.000,0.1310,0.3242,0.0000,4732
2,20230510,3.0,0.0361,0.3607,0.000,0.0835,0.5193,0.0000,5099
3,20230510,4.0,0.0218,0.1646,0.000,0.0467,0.7665,0.0000,5584
4,20230510,5 or more batches,0.0163,0.0737,0.000,0.0336,0.8760,0.0000,4911


In [66]:
#Metrics_pivot
metrics_pivot = pd.pivot_table(df_filter_level1, index = ['yyyymmdd'], columns = 'final_order_status',
 values = 'count', aggfunc = 'sum', fill_value=0).reset_index().rename_axis(None, axis=1)
metrics_pivot['total'] = metrics_pivot[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expiry_mapped', 'expiry_unmapped']].sum(axis = 1)
metrics_pivot['OCARA%'] = round(metrics_pivot['OCARA']/metrics_pivot['total'], 4)
metrics_pivot['COBRA%'] = round(metrics_pivot['COBRA']/metrics_pivot['total'], 4)
metrics_pivot['COBRM%'] = round(metrics_pivot['COBRM']/metrics_pivot['total'], 4)
metrics_pivot['G2N'] = round(metrics_pivot['dropped']/metrics_pivot['total'], 4)
metrics_pivot['expiry_mapped%'] = round(metrics_pivot['expiry_mapped']/metrics_pivot['total'], 4)
if 'expiry_unmapped' in metrics_pivot.columns:
    metrics_pivot['Stockout%'] = round(metrics_pivot['expiry_unmapped']/metrics_pivot['total'], 4)
else:
    metrics_pivot['Stockout%'] = 0
metrics_df = metrics_pivot[['yyyymmdd', 'OCARA%', 'COBRA%', 'COBRM%', 'G2N', 'expiry_mapped%', 'Stockout%', 'total']]
metrics_df

,yyyymmdd,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total
0,20230417,0.0504,0.3347,0.0000,0.1658,0.4485,0.0,32180
1,20230418,0.0500,0.3283,0.0001,0.1693,0.4516,0.0,32213


In [64]:
#Get the average ETA and FM
df_dropped = df_filter_level1[df_filter_level1['final_order_status'] == 'dropped']
fm = df_dropped.groupby(['yyyymmdd', 'counter_group'])['accept_to_pickup_distance'].median().reset_index()
fm = fm.rename(columns = {'accept_to_pickup_distance':'FM'})
eta = df_dropped.groupby(['yyyymmdd', 'counter_group'])['eta'].median().reset_index()
metrics_df = metrics_df.merge(fm, on = ['yyyymmdd', 'counter_group'], how = 'left')
metrics_df = metrics_df.merge(eta, on = ['yyyymmdd', 'counter_group'], how = 'left')
metrics_df.head()

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,20230417,1.0,0.0612,0.4615,0.0001,0.2649,0.2114,0.0001,10454,0.6850,3.0
1,20230417,2.0,0.0738,0.5051,0.0000,0.2289,0.1916,0.0000,4500,0.8165,3.0
2,20230417,3.0,0.0566,0.4134,0.0000,0.1649,0.3643,0.0000,4683,0.7500,3.0
3,20230417,4.0,0.0401,0.2120,0.0000,0.0772,0.6704,0.0000,5480,0.7570,3.0
4,20230417,5 or more batches,0.0234,0.0816,0.0000,0.0481,0.8467,0.0000,7063,0.7700,3.0


In [65]:
metrics_df

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,20230417,1.0,0.0612,0.4615,0.0001,0.2649,0.2114,0.0001,10454,0.6850,3.0
1,20230417,2.0,0.0738,0.5051,0.0000,0.2289,0.1916,0.0000,4500,0.8165,3.0
2,20230417,3.0,0.0566,0.4134,0.0000,0.1649,0.3643,0.0000,4683,0.7500,3.0
3,20230417,4.0,0.0401,0.2120,0.0000,0.0772,0.6704,0.0000,5480,0.7570,3.0
4,20230417,5 or more batches,0.0234,0.0816,0.0000,0.0481,0.8467,0.0000,7063,0.7700,3.0
5,20230418,1.0,0.0620,0.4518,0.0004,0.2787,0.2062,0.0001,10301,0.6870,3.0
6,20230418,2.0,0.0699,0.5101,0.0000,0.2138,0.2050,0.0000,4419,0.7970,3.0
7,20230418,3.0,0.0625,0.3872,0.0000,0.1792,0.3707,0.0000,4783,0.7330,3.0
8,20230418,4.0,0.0376,0.2154,0.0000,0.0821,0.6644,0.0000,5348,0.7980,3.0
9,20230418,5 or more batches,0.0223,0.0902,0.0000,0.0465,0.8409,0.0000,7362,0.7890,3.0


In [57]:
metrics_df

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM_x,eta_x,FM_y,eta_y
0,20230417,1.0,0.0612,0.4615,0.0001,0.2649,0.2114,0.0001,10454,3.404303,3.072228,0.6850,3.0
1,20230417,2.0,0.0738,0.5051,0.0000,0.2289,0.1916,0.0000,4500,16.989028,3.336893,0.8165,3.0
2,20230417,3 or more batches,0.0377,0.2133,0.0000,0.0891,0.6595,0.0000,17226,3.227829,3.340065,0.7570,3.0
3,20230418,1.0,0.0620,0.4518,0.0004,0.2787,0.2062,0.0001,10301,3.313941,3.058516,0.6870,3.0
4,20230418,2.0,0.0699,0.5101,0.0000,0.2138,0.2050,0.0000,4419,16.525852,3.299471,0.7970,3.0
5,20230418,3 or more batches,0.0380,0.2097,0.0000,0.0936,0.6584,0.0000,17493,7.587716,3.266178,0.7560,3.0


In [52]:
arr = [6, 5, 8, 10, 16, 17, 4, 2, 20]
arr.sort()
index = len(arr)//2
if len(arr)%2 == 0:
    print((arr[index - 1] + arr[index])/2)
else:
    print(arr[index])


8


In [55]:
x = 'mykhan'
y = [s for s in x]
y

['m', 'y', 'k', 'h', 'a', 'n']

In [56]:
df['x'].map(x_dict)

AttributeError: 'list' object has no attribute 'split'

In [ ]:
df['first_name', 'last_name'] = df['name'].split(' ')